<a href="https://colab.research.google.com/github/Zhi-704/ERL/blob/master/Genetic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install jumanji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 583.5/583.5 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for jumanji: filename=jumanji-0.3.1-py3-none-any.whl size=750375 sha256=38819a71f511bc9778ded47df8fa03b33990df8575c29e50bd1efa3c3ea1b794
  Stored in directory: /root/.cache/pip/wheels/31/29/10/884df072d319d2522cfff8b9c3d61fe60ab3c31e2a054d3e5d
Successfully built jumanji


In [3]:
import jumanji
import jax.numpy as jnp
import jax
import random
import tensorflow as tf
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Dense
import numpy as np
import matplotlib.pyplot as plt
from time import sleep

In [ ]:
# Useful Code
'''
# Instantiate tetris environment using registry

env = jumanji.make('Tetris-v0', num_rows = 5, time_limit = 1000)

# Reset your (jit-able) environment
key = jax.random.PRNGKey(1)
state, timestep = jax.jit(env.reset)(key)

def convert_state(self, state):
  '''
  # Convert state into observation variable that can be passed into neural network
  # PARAM -
  # state: current game state
  # RETURNS - observation variable
  '''

  grid = state.grid_padded.flatten().tolist()
  tetromino = state.tetromino_index.flatten().tolist()
  # print(grid)
  # print(tetromino)
  obs_variable = np.asarray(grid+tetromino)
  # print("Current input dimensions:")
  # print(np.shape(obs_variable))
  # print(obs_variable)

  return obs_variable

def policy(self,state):
  '''
  # Takes state from environment and returns an action that has the highest q value using epsilon_greedy
  # PARAM -
  # state: current game state
  # RETURNS - action
  '''
  # Matches state into 105, array
  state_array = self.convert_state(state)
  # Convert to into tensorflow tensor
  state_input = tf.convert_to_tensor(state_array[None, :], dtype=tf.float32)
  # Grabs Q values for all possible actions in current state
  action_q = self.q_net(state_input)

  action_mask = np.array(state.action_mask).flatten()

  valid_q_values = action_q * action_mask
  # Set elements that are zero to -999
  valid_q_values = np.where(valid_q_values == 0, -999, valid_q_values)

  print("ACTION MASK")
  print(action_mask)
  print("CORRESPONDING Q VALUES")
  print(valid_q_values)
  # print(np.shape(valid_q_values))
  true_indices = self.get_valid_actions(state)
  print(true_indices)
  if len(true_indices) == 0:
    return False, False
  else:
    action_index = np.argmax(valid_q_values, axis=1)[0]
    # Convert action into acceptable type
    array = np.array(state.action_mask)
    rotation, col_index = np.unravel_index(action_index, array.shape)
    action = [rotation, col_index]
    return action, action_index

def get_valid_actions(self,state):
  array = np.array(state.action_mask)
  # Flatten the array
  flattened_array = array.flatten()
  # Get the indices of true values, which are the number of valid actions in the current state
  true_indices = np.where(flattened_array)[0]

  return true_indices

def random_action(self, state):
  array = np.array(state.action_mask)
  true_indices = self.get_valid_actions(state)
  if len(true_indices) == 0:
    return False
  random_index = np.random.choice(true_indices)
  # Convert the random index to two-dimensional coordinates
  rotation, col_index = np.unravel_index(random_index, array.shape)
  return [rotation,col_index]
  '''

In [ ]:
'''
GA()
   initialize population
   find fitness of population

   while (termination criteria is reached) do
      parent selection
      crossover with probability pc
      mutation with probability pm
      decode and fitness calculation
      survivor selection
      find best
   return best
'''

In [81]:
class Population:

  def __init__(self, pop_size, p_mutation, p_crossover, p_selection, max_generations):
    self.pop_size = pop_size
    self.p_mutation = p_mutation
    self.p_crossover = p_crossover
    self.p_selection = p_selection
    self.max_generations = max_generations
    self.population = []
    self.elites = []

  def parent_crossover(parent_a, parent_b):

    offspring = Genome(105,64,40)

    # Get weight and biases of parents hidden layer
    weights1, biases1 = parent_a.model.layers[1].get_weights()
    weights2, biases2 = parent_b.model.layers[1].get_weights()

    # Swap weights between parents to create the offspring
    midpoint_w = len(weights1) // 2
    midpoint_b = len(biases1) // 2

    # print(midpoint_w)
    # print(midpoint_b)

    # Combine weights and biases from parents up to the midpoint
    offspring_weights = np.concatenate((weights1[:midpoint_w], weights2[midpoint_w:]), axis=0)
    offspring_biases = np.concatenate((biases1[:midpoint_b], biases2[midpoint_b:]), axis=0)

    offspring.model.layers[1].set_weights([offspring_weights, offspring_biases])

    return offspring

  def mutation(self, mutation_rate):

    # Choose random agent to mutate
    rand_num = random.randint(0, self.pop_size -1)
    # rand_num = 0
    current_genome = self.population[rand_num]

    # Chooses random weight to mutate
    model = current_genome.model
    hidden_layer = model.layers[1]  # Assuming hidden layer is the second layer

    weights, biases = hidden_layer.get_weights()

    for i in range(len(weights)):
      for j in range(len(weights[i])):
        if np.random.rand() < mutation_rate:
          # Apply mutation to the weight
          weights[i][j] += np.random.normal(-0.1, 0.1)  # You can adjust the mutation strategy here

    current_genome.model.layers[1].set_weights([weights, biases])
    return

  def selection(self):
    return

  def fitness(self):
    return

  def run(self):
    return

class Genome:
  def __init__(self, input_size, hidden_size, output_size):
    self.model = self.build_model(input_size, hidden_size, output_size)
    self.fitness = 0.0
    # self.weights_biases: np.array = None
    # weights = self.model.layers[1].get_weights()[0]
    # biases = self.model.layers[1].get_weights()[1]
    # print("Printing WEIGHTS")
    # print(weights)
    # print(len(weights))
    # print(weights[1])
    # print(weights[1][1])
    # print("Printing BIASES")
    # print(biases)
    # print(len(biases))
    # print(biases[1])

  def build_model(self, input_size, hidden_size, output_size):
    # 128, 64, 40 input, hidden,output

    # Vary the weights and biases for each genome
    kernel_init = tf.keras.initializers.RandomUniform(minval=-0.5, maxval=0.5)
    bias_init = initializer = tf.keras.initializers.RandomUniform(minval=-0.1, maxval=0.1)

    model = Sequential()
    # Adds fully connected layer with 128 units and uses rectified linear unit activation function. he_uniform initliazes weight of layer
    model.add(Dense(128, input_dim = input_size, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(hidden_size, activation='relu', use_bias=True, bias_initializer = bias_init, kernel_initializer= kernel_init))
    # 40 actions so 40 different outputs
    model.add(Dense(output_size, activation='linear', kernel_initializer='he_uniform'))

    # opt = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(loss='mse', optimizer = 'adam')
    return model

In [85]:
agent1 = Genome(105,64,40)
agent2 = Genome(105,64,40)

print("Starting population")
Big_poppa = Population(5,5,5,5,5)
offspring = Population.parent_crossover(agent1, agent2)
first_weights = offspring.model.layers[1].get_weights()[0]
print("Printing first weights")
print(first_weights)
Big_poppa.population.append(offspring)
Big_poppa.mutation(0.5)
second_weights = offspring.model.layers[1].get_weights()[0]
print("Printing second weights")
print(second_weights)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Starting population
Printing first weights
[[-0.3706503   0.16373265  0.29938018 ... -0.20711982  0.4458177
  -0.31277525]
 [-0.4483664  -0.35025764  0.28668022 ... -0.16248083  0.20240784
   0.06602335]
 [-0.09077668  0.17161334 -0.4261794  ... -0.04057741  0.3139943
   0.12084055]
 ...
 [-0.18130457 -0.27064764 -0.28934908 ... -0.02661538  0.15217805
  -0.18541873]
 [ 0.46953475  0.05670452 -0.4748088  ... -0.08701587  0.05308843
  -0.36621606]
 [-0.16469789  0.32211578  0.11526811 ... -0.17516863 -0.3273517
  -0.16775215]]
Printing second weights
[[-0.5169425  -0.01602344  0.29938018 ... -0.20711982  0.34304106
  -0.53519493]
 [-0.4483664  -0.56391585  0.28668022 ... -0.32966864  0.20240784
  -0.00442901]
 [-0.09077668  0.19065732 -0.5070619  ... -0.15665387  0.3139943
   0.11724859]
 ...
 [-0.37388808 -0.27064764 -0.28934908 ... -0.02661538 -0.10949036
  -0.18541873]
 [ 0.25781894  0.05670452 -0.4748088  ... -0.08701587  0.05308843
  -0.36745217]
 [-0.16469789  0.32211578  0.115268

In [ ]:
# https://github.com/robertjankowski/ga-openai-gym/blob/master/scripts/ga/individual.py

class Population:
  def __init__(self, individual, pop_size, max_generation, p_mutation, p_crossover, p_inversion):
    self.pop_size = pop_size
    self.max_generation = max_generation
    self.p_mutation = p_mutation
    self.p_crossover = p_crossover
    self.p_inversion = p_inversion
    self.old_population = [copy.copy(individual) for _ in range(pop_size)]
    self.new_population = []

In [ ]:
# https://www.youtube.com/watch?app=desktop&v=nhT56blfRpE&t=85s
from random import choices, randint, randrange, random
from typing import List, Optional, Callable, Tuple

Genome = List[int]

# Generate single genome for one solution
def generate_genome(length: int) -> Genome:
  return choices([0,1], k = length)

# Generate how every many genomes in a population
def generate_population(size: int, genome_length:int) -> Population:
  return [generate_genome(genome_length) for _ in range(size)]

def single_point_crossover(a: Genome, b: Genome) -> Tuple[Genome, Genome]:
  if len(a) != len(b):
      raise ValueError("Genomes a and b must be of same length")

  length = len(a)
  if length < 2:
      return a, b

  p = randint(1, length - 1)
  return a[0:p] + b[p:], b[0:p] + a[p:]


def mutation(genome: Genome, num: int = 1, probability: float = 0.5) -> Genome:
  for _ in range(num):
    index = randrange(len(genome))
    genome[index] = genome[index] if random() > probability else abs(genome[index] - 1)
  return genome


def population_fitness(population: Population, fitness_func: FitnessFunc) -> int:
  return sum([fitness_func(genome) for genome in population])


def selection_pair(population: Population, fitness_func: FitnessFunc) -> Population:
  return choices(
    population=population,
    weights=[fitness_func(gene) for gene in population],
    k=2
  )


def sort_population(population: Population, fitness_func: FitnessFunc) -> Population:
  return sorted(population, key=fitness_func, reverse=True)


def genome_to_string(genome: Genome) -> str:
  return "".join(map(str, genome))


def print_stats(population: Population, generation_id: int, fitness_func: FitnessFunc):
  print("GENERATION %02d" % generation_id)
  print("=============")
  print("Population: [%s]" % ", ".join([genome_to_string(gene) for gene in population]))
  print("Avg. Fitness: %f" % (population_fitness(population, fitness_func) / len(population)))
  sorted_population = sort_population(population, fitness_func)
  print(
      "Best: %s (%f)" % (genome_to_string(sorted_population[0]), fitness_func(sorted_population[0])))
  print("Worst: %s (%f)" % (genome_to_string(sorted_population[-1]),
                            fitness_func(sorted_population[-1])))
  print("")

  return sorted_population[0]


def run_evolution(
      populate_func: PopulateFunc,
      fitness_func: FitnessFunc,
      fitness_limit: int,
      selection_func: SelectionFunc = selection_pair,
      crossover_func: CrossoverFunc = single_point_crossover,
      mutation_func: MutationFunc = mutation,
      generation_limit: int = 100,
      printer: Optional[PrinterFunc] = None) \
      -> Tuple[Population, int]:
  population = populate_func()

  for i in range(generation_limit):
    population = sorted(population, key=lambda genome: fitness_func(genome), reverse=True)

    if printer is not None:
      printer(population, i, fitness_func)

    if fitness_func(population[0]) >= fitness_limit:
      break

    next_generation = population[0:2]

    for j in range(int(len(population) / 2) - 1):
      parents = selection_func(population, fitness_func)
      offspring_a, offspring_b = crossover_func(parents[0], parents[1])
      offspring_a = mutation_func(offspring_a)
      offspring_b = mutation_func(offspring_b)
      next_generation += [offspring_a, offspring_b]

    population = next_generation

  return population, i